#Corrección automática de examenes

En el presente trabajo se utilizará un dataset que consta de ocho grupos de "ensayos" con una extensión promedio de 150 a 550 palabras. El objetivo es aprender a utilizar texto como variable de entrada, el cual requiere una etapa de preprocesado diferente al utilizar otro tipo de variables (numéricas, categoricas, etc...). Esto se llevará a cabo implementando diferentes modelos de clasificación que permitan asignar una calificacion a cada "ensayo".

##Dataset

## Cargar dataset

Para cargar el dataset, utilizo la API de Kaggle. La cual requiere utilizar un archivo ".json" que permite descargar el dataset. Luego subí los archivos al repostiorio del proyecto para evitar repetir este proceso (subir .json y descargar todo el dataset) cada vez que utilice colab. 
Para descargar los datos cree el notebook [DownloadData_wAPI](https://github.com/GastonRAraujo/Materia-Ap_Maq/blob/master/Proyecto_Final/DownloadData_wAPI.ipynb), donde se encuentra el procedimiento paso a paso para descargar los archivos y subirlos a GitHub.

Una vez realizado esto, podemos acceder a ellos utilizando el link que nos otorga GitHub

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 4)

df_train = pd.read_csv('https://raw.githubusercontent.com/GastonRAraujo/Materia-Ap_Maq/master/Proyecto_Final/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')
df_test = pd.read_csv('https://raw.githubusercontent.com/GastonRAraujo/Materia-Ap_Maq/master/Proyecto_Final/valid_set.csv', sep=',', encoding='ISO-8859-1')

AttributeError: module 'pandas' has no attribute 'set_option'

In [ ]:
df_train.head(3)

In [ ]:
df_test.head(3)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
print(df_train.groupby('essay_set')['essay_id'].nunique().values)

print(df_test.groupby('essay_set')['essay_id'].nunique().values)

En ambos casos el grupo 8 se ve subrepresentado.

Oservamos una gran presencia de valores null en algunas columnas, si investigamos sobre el dataset podemos ver el significado de cada una y poque sucede esto:


  * essay_id: un identificador único para cada ensayo individual del estudiante

  * essay_set: 1-8, una identificación para cada conjunto de ensayos

  * essay: el texto ascii de la respuesta de un estudiante

  * rater1_domain1: puntuación del dominio 1 del evaluador 1; todos los ensayos tienen esto

  * rater2_domain1: puntuación del dominio 1 del evaluador 2; todos los ensayos tienen esto

  * rater3_domain1: puntuación del dominio 1 del evaluador 3; solo algunos ensayos del conjunto 8 tienen esto.

  * domain1_score: puntuación resuelta entre los evaluadores; todos los ensayos tienen esto

  * rater1_domain2: puntuación del dominio 2 del evaluador 1; solo los ensayos del conjunto 2 tienen esto

  * rater2_domain2: puntuación del dominio 2 del evaluador 2; solo los ensayos del conjunto 2 tienen esto

  * domain2_score: puntuación resuelta entre los evaluadores; solo los ensayos del conjunto 2 tienen esto

  * rater1_trait1 score - rater3_trait6 score: puntajes de rasgos para los conjuntos 7-8


##Procesamiento de datos

En esta sección procesaremos los datos por lo que analizaremos como reducir la dimension de nuestro dataset, como tratar los valores NaN y por último realizaremos el procesamiento de texto.


###Procesamiento de variables númericas

Analicemos la distribucion de notas de cada grupo

In [ ]:
import plotly.express as px

fig = px.histogram(df_train, x="domain1_score", color="essay_set", marginal="rug", hover_data=None)
fig.show()

Los grupos poseen diferentes distribuciones de notas. Mientras la mayoría se encuentra con una media menor a 30 puntos, el grupo 8 posee una distribución con notas más altas.
A su vez se observa que una gran cantidad de alumnos a obtenido una calificación menor a 6.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Correlation matrix
plt.figure(dpi=150)
 
sns.heatmap(df_train.corr(), vmin=-1, vmax=1, center= 0, xticklabels=True, yticklabels=True)
 
#Adjust font size
sns.set(font_scale=0.7)
 
# Rotate the tick labels a bit
plt.xticks(rotation=85)
 
plt.show()

Se observa una correlación entre algunas columnas, lo cual surge de que las columnas "rater#_trait#" contienen la información con la que cada evualdor calificó finalmente al alumno. Por lo que un alumno que obtenga altos puntajes en los diferentes rasgos, obtendrá un puntaje alto según ese evaluador. A su vez vemos que estos rasgos estan fuertemente relacionados entre si.
En principio pareciera que "domain1_score" es simplemente la suma de las calificaciones. Comprobemos que "domain1_score" NO es la suma de las calificaciones que le han otorgado.

In [ ]:
dummy_df = df_train[df_train['domain1_score']!=df_train['rater1_domain1']+df_train['rater2_domain1']]

dummy_df

In [ ]:
dummy_df = df_train[df_train['rater3_domain1']!=df_train['domain1_score']]

dummy_df

Por lo que la columna "domain1_score" no es la suma ni el promedio de las columnas "rater1_domain1",	"rater2_domain1" y	"rater3_domain1". Hay una similitud en "domain1_score" y "rater3_domain1" por lo que descartaremos esta última, además de que la amplia mayoría no cuenta con un valor para esta característica.

In [ ]:
dummy_df = df_train[df_train['rater1_trait1']>0]

dummy_df

Las columnas "rater#_trait#" nos dan información de como obtuvieron el puntaje de "rater#_domain1" por lo que no las utilizaremos.

Repitamos el analisis pero para el "domain2"

In [ ]:
dummy_df = df_train[df_train['domain2_score']>0]

dummy_df

Observamos que nuevamente la columna "domain1_score" no corresponde a la suma ni al promedio de las columnas de "rater#_domain2".

A continuacion eliminamos las columnas que no utilizaremos y unificaremos ambos puntajes "domain#_score" realizando un promedio de ambos.

In [ ]:
# Calculamos el promedio de puntajes ignorando NaN
# Es decir que el promedio de 5 y Nan es 5, pero 5 y 2 es (5+2)/2
df_train['score'] = df_train[['domain1_score', 'domain2_score']].mean(axis=1)


# Eliminamos las columnas que no utilizaremos:

#Eliminamos aquellas que tengan valores NaN
df_train.dropna(axis='columns',inplace=True)

#Eliminamos las restantes
df_train.drop(['rater1_domain1',	'rater2_domain1',	'domain1_score'], axis=1, inplace=True)

fig = px.histogram(df_train, x="score", color="essay_set", histnorm='probability density')
fig.show()

In [ ]:
df_train.drop(['essay_id',	'essay_set'], axis=1, inplace=True)

fig = px.histogram(df_train, x="score", histnorm='probability density')
fig.show()

Observamos la misma distribución que al comienzo solo que los grupos han sido unificados.

Para los datos de validación solo hace falta eliminar las columnas correspondientes

In [ ]:
# Eliminamos las restantes
df_test.drop(['essay_id',	'essay_set'], axis=1, inplace=True)

df_test

In [ ]:
fig = px.histogram(df_test, x="predicted_score", histnorm='probability density')
fig.show()

###Procesamiento de texto

Para procesar el texto y poder usarlo como input de nuestro modelo debemos realizar los siguientes pasos:

* *Tokenizar* el texto: convertir el texto en lista de palabras
* Cambiar mayúsculas por minúsculas
* Remover *stop words* (palabras que no aportan información: the, is, at, which, on, for, this, etc.) y signos de punctuación
* Stemming: método para reducir una palabra a su raíz 
* Lematización: proceso lingüístico que consiste en, dada una forma flexionada (es decir, en plural, en femenino, conjugada, etc), hallar el lema correspondiente. Por ejemplo decir es el lema de dije
* Vectorización: convertir nuestro texto tokenizado y procesado en vectores

####Tokenizacion y prepocesamiento

In [ ]:
import numpy as np
import re # expresiones regulares
import nltk #procesar texto

#descargas necesarias
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# Procesamiento de texto
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet #tipo de palabra
from nltk import word_tokenize #tokenizer

import string  

La siguiente función elimina las signos de puntuación, convierte mayúsculas, remueve URLS, tokeniza por minúsculas y por último aplica un stemmer que reduce las palabras a su raiz. Esto puede generar que algunas palabras sean cortadas o pierdan caracteres por lo que luego se aplica un lematizador que 

In [ ]:
# Stemmer 
stemmer = nltk.stem.PorterStemmer()

# creo una lista de signos de puntuación que serán eliminados
Punct_Sign = string.punctuation

# Set de stopwords del idioma ingles
stopwords_english = set(stopwords.words('english'))

# Utilizo expresiones regulares para quitar URLs
url_pattern = re.compile(r'https?://\S+|www\.\S+')


def clean_text(text):
    # Cambiar Mayúscula por minúscula
    text = text.lower()

    # Remuevo signos de puntuación
    # maketrans(x,y,z) cambia "x" por "y" y remueve "z"
    text = text.translate(str.maketrans('', '', Punct_Sign)) #no hago cambios pero remuevo signos de puntuacion

    # Remuevo links
    text = url_pattern.sub(r'', text)

    # StopWords
    text = [stemmer.stem(word) for word in str(text).split() if word not in stopwords_english]

    return text

In [ ]:
df_train["text"] = df_train["essay"].apply(lambda text: clean_text(text))
df_train.head()

El stemmer puede suceder que elimine letras o que no encuentre correctamente la raiz como en el caso de "having". Esto se debe a que se centra en ser un algoritmo rápido pero que no tiene en cuenta el contexto ni utiliza una base de datos de léxico, no podría diferenciar dos palabras con significados similares/distintos como el caso de "better" que posee "good" como lema pero "bet" como raiz. Se perdería la conexión entre ambas entidades. Por lo que cambiaremos el stemmer por el lematizador, al cual hay que indicarle el tipo de palabra que debe procesar debido a que una misma palabra puede ser un verbo o un sustantivo y poseer diferente raiz según el caso.

In [ ]:
# Lemmatizer 
lemmatizer = nltk.stem.WordNetLemmatizer()

# Dic con KeyWord: tipo de palabra
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

# creo una lista de signos de puntuación que serán eliminados
Punct_Sign = string.punctuation

# Set de stopwords del idioma ingles
stopwords_english = set(stopwords.words('english'))

# Utilizo expresiones regulares para quitar URLs
url_pattern = re.compile(r'https?://\S+|www\.\S+')


def clean_text(text):
    # Cambiar Mayúscula por minúscula
    text = text.lower()

    # Remuevo signos de puntuación
    # maketrans(x,y,z) cambia "x" por "y" y remueve "z"
    text = text.translate(str.maketrans('', '', Punct_Sign)) #no hago cambios pero remuevo signos de puntuacion

    # Remuevo links
    text = url_pattern.sub(r'', text)
      
    # StopWords
    text = [word for word in str(text).split() if word not in stopwords_english]
    
    #   # Lematización
    pos_tagged_text = nltk.pos_tag(text)
    text = " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

    return text

In [ ]:
!pip install swifter

In [ ]:
import swifter

df_train["text"] = df_train["essay"].swifter.apply(lambda text: clean_text(text))

In [ ]:
%time df_train["text"] = df_train["essay"].apply(lambda text: clean_text(text))
df_train.drop(columns="essay")
df_train.head()

In [ ]:
df_test["text"] = df_test["essay"].apply(lambda text: clean_text(text))
df_test.drop(columns="essay")
df_test.head()

Si bien el proceso es más lento, en este caso particular, obtenemos un mejor resultado con el lematizador. Se debe utilizar el procedimiento correspondiente según los textos a procesar.

En algunos casos es conveniente eliminar las palabras más comunes si estas no aportan información útil. Para eso debemos comprobar que no eliminaremos paralbras que si aporten información útil.

In [ ]:
#Palabras más comunes
from collections import Counter

#Contador para datos de entrenamiento
cnt = Counter()
for text in df_train["text"].values:
    for word in text:
        cnt[word] += 1

print(cnt.most_common(10))

#Contador para datos de validación
cnt2 = Counter()
for text in df_test["text"].values:
    for word in text:
        cnt2[word] += 1

print(cnt2.most_common(10))

En ambos conjutos de datos (validación y entrenamiento), las palabras más comunes coinciden en su mayoría. 

Al igual que con las palabras más comunes, podemos realizar el mismo analisis para las palabras más raras.

In [ ]:
n_rare_words = 10
rarewords = set([(w,wc) for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
print(rarewords)

rarewords2 = set([(w,wc) for (w, wc) in cnt2.most_common()[:-n_rare_words-1:-1]])
print(rarewords2)

Podemos observar que no solo son palabras que no debemos eliminar sino que no coinciden en ambos casos. Podemos realizar una nube de palabras para ambos conjuntos de datos y así poder comparar las palabras más y menos frecuentes de ambos.

In [ ]:
from wordcloud import WordCloud

a = [' '.join(map(str, l)) for l in df_train['text']]

word_cloud = WordCloud(
                       width=1600,
                       height=800,
                       background_color="white",collocations=False
            ).generate(" ".join(a))

plt.figure(figsize=(16,8))
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
a = [' '.join(map(str, l)) for l in df_test['text']]

word_cloud = WordCloud(
                       width=1600,
                       height=800,
                       margin=0,
                       background_color="white",collocations=False
            ).generate(" ".join(a))

plt.figure(figsize=(16,8))
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

####Vectorización
Por último convertiremos el texto ya procesado y tokenizado en vector. Para ello utilizaremos el método [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) de scikit-learn. El cual construye una transformación de tokens a vectores a partir del vocabulario de nuestros datos.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Creo una lista con todos los textos de entrenamiento
text_train    = [''.join(x) for x in df_train["text"].values ] # train texts
# Creo una lista con todos los textos de validación
text_test    = [''.join(x) for x in df_test["text"].values ] # val texts

# Unifico ambas para crear una lista con todos los textos
# para entrenar el vectorizador
texts = text_train + text_test

In [ ]:
# Vectorizador
vectorizer = CountVectorizer() 
vectorizer = vectorizer.fit(texts) #utilizo todos los textos
print("Vocabulary size:{}".format(len(vectorizer.vocabulary_)))
print("Vocabulary content:{}".format(vectorizer.vocabulary_))

In [ ]:
X_train = vectorizer.transform(df_train['text'])
y_train = df_train['score']

X_test = vectorizer.transform(df_test['text'])
y_test = df_test['predicted_score']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer

tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
train_tfidf = tfidf.fit_transform(df_train['text'])
test_tfidf = tfidf.transform(df_test['text'])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
import collections

In [ ]:
# Let's implement simple classifiers

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(n_neighbors=1),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "MultinimialNB": MultinomialNB()
}

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

classifier = KNeighborsClassifier()

kappa = make_scorer(cohen_kappa_score, weights='quadratic')

classifier.fit(X_train, df_train["score"].astype(int))
training_score = cross_val_score(classifier, X_train, df_train["score"].astype(int), cv=5, scoring=kappa )
print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

In [ ]:
# Using the Logistic Regression

classifier2 = LogisticRegression()

classifier2.fit(X_train, df_train["score"].astype(int))
training_score = cross_val_score(classifier2, X_train, df_train["score"].astype(int), cv=5, scoring=kappa)
print("Classifiers: ", classifier2.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

In [ ]:
# Using the XGBoost

import xgboost as xgb
clf_xgb = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=3000,
    max_depth=15,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    nthread=42,
    scale_pos_weight=1,
    seed=27)

scores = model_selection.cross_val_score(clf_xgb, X_train, df_train["score"].astype(int), cv=5, scoring=kappa)

##Modelos 

Se entrenarán varios modelos de clasificadores desde los más simples: Naive Bayes y Kmeans hasta modelos más complejos de *DeepLearning* utilisando una red neuronal secuencial con capas LTSM.

##resultados y conclusiones